# AI Tutor - Textbook Ingestion (Google Colab)

This notebook uses the AI Tutor's ingestion system to process textbooks.

**Features:**
- ✅ Uses TutorSystem ingestion pipeline
- ✅ Upload your own PDFs or use sample textbooks
- ✅ Creates embeddings with sentence-transformers
- ✅ Works with `chroma_data/chroma.sqlite3` database
- ✅ Save/load database from Google Drive


In [ ]:
print("🔄 Cloning AI Tutor repository...")
!git clone https://github.com/HenryNVP/ai-tutor.git
%cd ai-tutor
!pip install -r requirements.txt

In [ ]:
import os
from pathlib import Path

PROJECT_ROOT = Path.cwd()
print(f"Project root: {PROJECT_ROOT}")

# Add src to Python path
import sys
SRC_ROOT = PROJECT_ROOT / "src"
if str(SRC_ROOT) not in sys.path:
    sys.path.insert(0, str(SRC_ROOT))

print("✅ Dependencies installed")

In [ ]:
# Initialize TutorSystem with chroma_data directory
from ai_tutor.system import TutorSystem
from ai_tutor.config.loader import load_settings
from pathlib import Path
import os
from google.colab import userdata

# Optional: Set API key
os.environ["OPENAI_DEFAULT_MODEL"]="gpt-4o-mini"
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

print("🚀 Loading settings...")
settings = load_settings()

# Override vector_store_dir to use chroma_data instead of data/vector_store
settings.paths.vector_store_dir = Path("chroma_data")
print(f"  Vector store directory: {settings.paths.vector_store_dir}")

# Create chroma_data directory if it doesn't exist
chroma_data_dir = PROJECT_ROOT / "chroma_data"
chroma_data_dir.mkdir(exist_ok=True)
print(f"  Created/verified chroma_data directory: {chroma_data_dir}")

print("\n🚀 Initializing TutorSystem...")
system = TutorSystem(settings=settings)

print(f"\n✓ TutorSystem initialized")
print(f"  Embedding model: {system.embedder.config.model}")
print(f"  Chunk size: {system.settings.chunking.chunk_size}")
print(f"  Vector store: {system.settings.paths.vector_store_dir}")
print(f"  Database location: {chroma_data_dir}/chroma.sqlite3")


In [ ]:
from google.colab import drive

drive.mount('/content/drive')
# Data folder in Drive
drive_folder_path = '/content/drive/MyDrive/ai-tutor/raw'

In [ ]:
# Upload PDFs
from pathlib import Path
from google.colab import files
import shutil

raw_data_dir = PROJECT_ROOT / "data" / "raw"
raw_data_dir.mkdir(parents=True, exist_ok=True)

# Copy PDF files from Google Drive to data/raw
print(f"Copying PDF files from '{drive_folder_path}' to '{raw_data_dir}'...")
for pdf_file in Path(drive_folder_path).glob("*.pdf"):
    shutil.copy(pdf_file, raw_data_dir / pdf_file.name)
    print(f"✅ {pdf_file.name}")

# List all PDF files
pdf_files = sorted(raw_data_dir.glob("*.pdf"))
print(f"\n📚 Found {len(pdf_files)} PDF file(s) ready for ingestion:\n")
for i, pdf in enumerate(pdf_files, 1):
    size_mb = pdf.stat().st_size / (1024 * 1024)
    print(f"  {i}. {pdf.name} ({size_mb:.1f} MB)")

In [ ]:
print("🔧 Fixing transformers compatibility...")
!pip install --upgrade 'transformers>=4.46.0,<5.0.0' --quiet
!pip install --upgrade --force-reinstall sentence-transformers --quiet
print("✅ transformers and sentence-transformers updated")

In [ ]:
# Reload modules to ensure changes take effect
import importlib
import sys
modules_to_reload = [m for m in sys.modules.keys() if 'transformers' in m or 'sentence_transformers' in m]
for module in modules_to_reload:
    if module in sys.modules:
        del sys.modules[module]

# Verify the fix
try:
    from transformers.trainer import Trainer
    print("✅ transformers.trainer import test passed")


In [ ]:
# Ingest all textbooks
# This will:
# 1. Parse PDFs
# 2. Chunk the text
# 3. Generate embeddings
# 4. Store in vector store

print("🔄 Starting ingestion (this may take a few minutes)...\n")

result = system.ingest_directory(raw_data_dir)

print("\n" + "="*60)
print("✅ INGESTION COMPLETE")
print("="*60)
print(f"📄 Documents processed: {len(result.documents)}")
print(f"📝 Chunks created: {len(result.chunks)}")
print(f"⏭️  Files skipped: {len(result.skipped)}")

if result.skipped:
    print(f"\nSkipped files:")
    for skip in result.skipped:
        print(f"  - {skip}")


In [ ]:
# Verify ingestion
print("📊 Verification:\n")

# Check chunks file
chunks_file = PROJECT_ROOT / "data" / "processed" / "chunks.jsonl"
if chunks_file.exists():
    with open(chunks_file, 'r') as f:
        chunk_count = sum(1 for _ in f)
    print(f"✓ Chunks stored: {chunk_count}")
else:
    print("✗ No chunks file found")

# Check Chroma database
chroma_db_path = PROJECT_ROOT / "chroma_data" / "chroma.sqlite3"
if chroma_db_path.exists():
    import sqlite3
    conn = sqlite3.connect(chroma_db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    conn.close()
    print(f"✓ Chroma database exists: {chroma_db_path}")
    print(f"  Tables: {len(tables)}")
else:
    print("✗ No Chroma database found")

# Check for collection directories
chroma_data_dir = PROJECT_ROOT / "chroma_data"
collection_dirs = [d for d in chroma_data_dir.iterdir() if d.is_dir() and d.name != 'chroma_example']
if collection_dirs:
    print(f"✓ Collection directories: {len(collection_dirs)}")
else:
    print("ℹ️  No collection directories yet (will be created on first use)")

print("\n✅ Ingestion successful!")


In [ ]:
folders_to_download = [
    PROJECT_ROOT / "data" / "processed",
    PROJECT_ROOT / "data" / "vector_store"
]

for folder_to_download in folders_to_download:
    if folder_to_download.exists():
        zip_filename = f"{folder_to_download.name}.zip"
        # Create a zip archive of the folder
        print(f"Compressing '{folder_to_download}' into '{zip_filename}'...")
        shutil.make_archive(zip_filename.replace(".zip", ""), 'zip', folder_to_download)

        # Download the zip file
        print(f"Downloading '{zip_filename}'...")
        files.download(zip_filename)
        print(f"✅ Download initiated for {zip_filename}")
    else:
        print(f"❗️ Folder not found: {folder_to_download}. Skipping download for this folder.")

print("\nFinished processing download requests.")

In [ ]:
# Save database to Google Drive
chroma_db_path = PROJECT_ROOT / 'chroma_data' / 'chroma.sqlite3'
drive_db_dir = Path('/content/drive/MyDrive/ai-tutor/chroma_data')
drive_db_dir.mkdir(parents=True, exist_ok=True)
drive_db_path = drive_db_dir / 'chroma.sqlite3'

if chroma_db_path.exists():
    print(f"💾 Saving database to Drive...")
    shutil.copy(chroma_db_path, drive_db_path)
    size_mb = chroma_db_path.stat().st_size / (1024 * 1024)
    print(f"✅ Database saved to Drive")
    print(f"   Size: {size_mb:.2f} MB")
    print(f"   Location: {drive_db_path}")
    
    # Also save collection directories if they exist
    chroma_data_dir = PROJECT_ROOT / 'chroma_data'
    for item in chroma_data_dir.iterdir():
        if item.is_dir() and item.name != 'chroma_example':
            drive_collection_dir = drive_db_dir / item.name
            if drive_collection_dir.exists():
                shutil.rmtree(drive_collection_dir)
            shutil.copytree(item, drive_collection_dir)
            print(f"   ✅ Saved collection directory: {item.name}")
else:
    print("⚠️  No database found to save")
